# Multinomial Logistic Regression Model

Attempting to predict underlying cause of death from demographic features.

## Setup

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
.master("local[*]") \
.appName("spark_setup") \
.getOrCreate()

In [2]:
# Read in train data
train_data = spark.read.option("header",True).option("inferSchema",True).csv("train_data")

In [3]:
# Subset with potential features
training = train_data.select("ucd", "education", "sex", "age", "marital_status", "hispanic_race_recode")

In [4]:
training.printSchema()

root
 |-- ucd: string (nullable = true)
 |-- education: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- hispanic_race_recode: integer (nullable = true)



In [5]:
# strip UCD codes to include first letter only, for response level reduction
from pyspark.sql.functions import *

training = training.withColumn("ucd_short", regexp_replace('ucd', '\\d+', ''))

In [6]:
training.show(5)

+----+---------+---+---+--------------+--------------------+---------+
| ucd|education|sex|age|marital_status|hispanic_race_recode|ucd_short|
+----+---------+---+---+--------------+--------------------+---------+
| F03|       14|  F| 94|             W|                   6|        F|
|J449|       12|  F| 70|             M|                   6|        J|
|I219|       16|  M| 88|             M|                   6|        I|
|C259|       12|  M| 77|             M|                   6|        C|
|F102|       12|  F| 46|             D|                   1|        F|
+----+---------+---+---+--------------+--------------------+---------+
only showing top 5 rows



In [7]:
# replace age coding values

def replace(column, value):
    return when(column != value, column)

training = training.withColumn("age", replace(col("age"), 999)) # change age value coded as unknown (999) to null

In [8]:
training.filter(training.age == 999).count() # yes - 999 codes have been replaced 

0

In [9]:
training.groupby("ucd_short").count().orderBy('count', ascending=False).show(5) # top 5 causes of death with counts

+---------+------+
|ucd_short| count|
+---------+------+
|        I|674836|
|        C|477413|
|        J|216611|
|        G|159912|
|        F|111510|
+---------+------+
only showing top 5 rows



I: Diseases of the circulatory system  
C: Neoplasms  
J: Diseases of the respiratory system  
G: Diseases of the nervous system  
F: Mental, Behavioral and Neurodevelopmental disorders  

In [10]:
training.select([count(when(col(c).isNull(), c)).alias(c) for c in training.columns]).show() # rows with null values in each column

+---+---------+---+---+--------------+--------------------+---------+
|ucd|education|sex|age|marital_status|hispanic_race_recode|ucd_short|
+---+---------+---+---+--------------+--------------------+---------+
|  0|    54142|  0|382|         16592|                7871|        0|
+---+---------+---+---+--------------+--------------------+---------+



In [11]:
training.count() # rows in training set

2200579

In [12]:
674836/2200579

0.30666292825660885

30.7% of deaths in the training set are due to circulatory disease.

In [13]:
# drop rows with null or nans
training = training.na.drop()

In [14]:
training.count()

2136639

In [15]:
(1 - (2136639/2200579))*100 # dropped 2.9% of dataset because of null values

2.9055989355528666

## Prep and parse the data

In [16]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml import Pipeline

In [17]:
train_set, val_set = training.randomSplit(weights=[0.8, 0.2], seed=212) ## split into train set and validation set

In [18]:
train_set.show()

+----+---------+---+---+--------------+--------------------+---------+
| ucd|education|sex|age|marital_status|hispanic_race_recode|ucd_short|
+----+---------+---+---+--------------+--------------------+---------+
|A020|        7|  M| 70|             M|                   6|        A|
|A021|        4|  M| 19|             S|                   7|        A|
|A021|        5|  M| 30|             S|                   6|        A|
|A021|        7|  M| 85|             W|                   6|        A|
|A021|       12|  F| 58|             M|                   6|        A|
|A047|        1|  F| 61|             S|                   7|        A|
|A047|        1|  F| 69|             W|                   6|        A|
|A047|        1|  F| 70|             D|                   1|        A|
|A047|        1|  F| 70|             S|                   6|        A|
|A047|        1|  F| 71|             S|                   4|        A|
|A047|        1|  F| 71|             W|                   6|        A|
|A047|

In [19]:
train_set.groupby("ucd_short").count().sort('count').show()

+---------+------+
|ucd_short| count|
+---------+------+
|        U|     2|
|        H|    90|
|        O|   721|
|        L|  3015|
|        Q|  5905|
|        P|  6371|
|        Y|  8497|
|        M|  8533|
|        B| 10081|
|        D| 16769|
|        R| 19986|
|        V| 25491|
|        W| 29747|
|        A| 33134|
|        N| 43401|
|        K| 65293|
|        E| 76540|
|        X| 78864|
|        F| 86364|
|        G|125106|
+---------+------+
only showing top 20 rows



In [20]:
# there are only 2 observations for group 'U' and 90 for group 'H' - filter out for now 

train_set = train_set.filter(train_set.ucd_short != "U")
train_set = train_set.filter(train_set.ucd_short != "H")

In [21]:
train_set.groupby("ucd_short").count().sort('count').show()

+---------+------+
|ucd_short| count|
+---------+------+
|        O|   721|
|        L|  3015|
|        Q|  5905|
|        P|  6371|
|        Y|  8497|
|        M|  8533|
|        B| 10081|
|        D| 16769|
|        R| 19986|
|        V| 25491|
|        W| 29747|
|        A| 33134|
|        N| 43401|
|        K| 65293|
|        E| 76540|
|        X| 78864|
|        F| 86364|
|        G|125106|
|        J|168089|
|        C|372398|
+---------+------+
only showing top 20 rows



In [22]:
train_set.select('ucd').distinct().count()

3465

In [23]:
train_set.select('ucd_short').distinct().count()

21

There are now 21 different outcomes for UCD.

## Modeling

### Full Training Set

In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexer1 = StringIndexer(inputCol="marital_status", outputCol="marital_status_index")
indexer2 = StringIndexer(inputCol="sex", outputCol="sex_index")
indexer3 = StringIndexer(inputCol="ucd_short", outputCol="ucd_short_index")

indexed1_full = indexer1.fit(train_set).transform(train_set)
indexed2_full = indexer2.fit(indexed1_full).transform(indexed1_full)
indexed_full = indexer3.fit(indexed2_full).transform(indexed2_full)

# OneHotEncoder transformer to prepare categorical features for model
encoder = OneHotEncoderEstimator(inputCols =["hispanic_race_recode", "education", "marital_status_index"],
                                 outputCols = ["cat_hispanic_race_recode", "cat_education", "cat_marital_status"])

model_full = encoder.fit(indexed_full)
encoded_full = model_full.transform(indexed_full)

# VectorAssembler to package the features for model
assembler_full = VectorAssembler(inputCols=["cat_hispanic_race_recode", "cat_education", "cat_marital_status", "sex_index", "age"],
                            outputCol="features")
output_full = assembler_full.transform(encoded_full)

cleanup_full = output_full.select("ucd_short_index", "features")
cleanup_full.printSchema()

mlr_full = LogisticRegression(featuresCol = "features", labelCol = "ucd_short_index", family = "multinomial")

mlrModel_full = mlr_full.fit(cleanup_full)

prediction_full = mlrModel_full.transform(cleanup_full)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [26]:
prediction_full.show()

+---------------+--------------------+--------------------+--------------------+----------+
|ucd_short_index|            features|       rawPrediction|         probability|prediction|
+---------------+--------------------+--------------------+--------------------+----------+
|            9.0|(30,[6,15,25,29],...|[3.43064048346362...|[0.26935644028785...|       1.0|
|            9.0|(30,[7,12,29],[1....|[1.54206074263952...|[0.04498454519431...|       5.0|
|            9.0|(30,[6,13,29],[1....|[1.86970683294612...|[0.07370801633457...|       5.0|
|            9.0|(30,[6,15,26,29],...|[4.04046433139016...|[0.37211904570899...|       0.0|
|            9.0|(30,[6,20,25,28,2...|[2.83668530956359...|[0.21191693039509...|       1.0|
|            9.0|(30,[7,9,28,29],[...|[2.74034622258615...|[0.31689565656866...|       0.0|
|            9.0|(30,[6,9,26,28,29...|[3.08447452699901...|[0.29857295948084...|       0.0|
|            9.0|(30,[1,9,27,28,29...|[3.03111893643889...|[0.29885561696088...|

### Evaluation: Full Training Set

In [27]:
# evaluation
evaluator = MulticlassClassificationEvaluator(
    labelCol="ucd_short_index", predictionCol="prediction", metricName="accuracy")

In [28]:
accuracy = evaluator.evaluate(prediction_full)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 34.55423957482799%
Test Error: 65.44576042517201%


Accuracy for the base model is 34.6%.

In [29]:
prediction_full.groupby("prediction").count().show()

+----------+-------+
|prediction|  count|
+----------+-------+
|       0.0|1227017|
|      18.0|    536|
|       1.0| 383334|
|       4.0|    636|
|      11.0|      2|
|       3.0|    103|
|       2.0|     41|
|      17.0|  17709|
|       5.0|  79081|
|      12.0|     26|
+----------+-------+



In [30]:
prediction_full.count()

1708485

In [31]:
1227086 / 1708485

0.7182304790501526

71.8% of predictions were made for Circulatory Disease.  This is because the dataset is unbalanced.  Will attempt downsampling and rerun the model to see if predictions are more evenly distributed.

### Full Downsampled Training Set

In [32]:
# function to do downsampling

def downSample(df, target, seed):
    
    # gather counts of each class 
    class_counts = df.groupby(target).count()

    # select smallest count size and corresponding class
    smallest_class_size = class_counts.agg({'count': 'min'})
    smallest_class_size = smallest_class_size.collect()[0]['min(count)']

    # generate ratio of each class to smallest class - for use with .sample()
    class_counts = class_counts.withColumn('min', lit(smallest_class_size))
    class_counts = class_counts.withColumn('ratio', class_counts['min']/ class_counts['count'])

    smallest_class = class_counts.filter(class_counts['count'] == class_counts['min']).collect()[0][target]
    
    # set up final dataframe to hold results - with only the smallest class to start
    adjusted_df = df.filter(df[target] == smallest_class)

    # iterate over outcome classes, sampling to match count of smallest class
    for i in range(class_counts.count()):

        outcome_class = class_counts.collect()[i][target]
        ratio = class_counts.collect()[i]['ratio']

        if outcome_class != smallest_class: 

            subset = df.filter(df[target] == outcome_class)
            subset_adjusted = subset.sample(False, ratio, seed = seed)

            adjusted_df = adjusted_df.unionAll(subset_adjusted)
            
        else:
            adjusted_df = adjusted_df

    return adjusted_df

In [33]:
adj_train = downSample(df = train_set, target = 'ucd_short', seed = 212)

In [34]:
adj_train.groupby("ucd_short").count().sort('count').show()

+---------+-----+
|ucd_short|count|
+---------+-----+
|        L|  676|
|        B|  691|
|        V|  691|
|        W|  699|
|        D|  700|
|        R|  705|
|        J|  707|
|        C|  709|
|        P|  713|
|        I|  716|
|        Y|  716|
|        Q|  719|
|        M|  720|
|        A|  721|
|        K|  721|
|        O|  721|
|        E|  733|
|        N|  734|
|        G|  734|
|        X|  739|
+---------+-----+
only showing top 20 rows



In [35]:
# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexed1_adj = indexer1.fit(adj_train).transform(adj_train)
indexed2_adj = indexer2.fit(indexed1_adj).transform(indexed1_adj)
indexed_adj = indexer3.fit(indexed2_adj).transform(indexed2_adj)

# One-Hot Encoder
model_adj = encoder.fit(indexed_adj)
encoded_adj = model_adj.transform(indexed_adj)

# VectorAssembler to package the features for model
output_adj = assembler_full.transform(encoded_adj)

# Cleanup
cleanup_adj = output_adj.select("ucd_short_index", "features")
cleanup_adj.printSchema()

# Model
mlr_adj = LogisticRegression(featuresCol = "features", labelCol = "ucd_short_index", family = "multinomial")
mlrModel_adj = mlr_adj.fit(cleanup_adj)
prediction_adj = mlrModel_adj.transform(cleanup_adj)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [36]:
prediction_adj.show()

+---------------+--------------------+--------------------+--------------------+----------+
|ucd_short_index|            features|       rawPrediction|         probability|prediction|
+---------------+--------------------+--------------------+--------------------+----------+
|            7.0|(30,[7,11,29],[1....|[-1.5409515579916...|[0.00346895753445...|       7.0|
|            7.0|(30,[7,12,27,29],...|[-2.0140562186020...|[0.00241958355446...|       7.0|
|            7.0|(30,[6,11,27,29],...|[-1.9645712464057...|[0.00298815800337...|       7.0|
|            7.0|(30,[7,11,25,29],...|[-2.0323252344512...|[8.80738053847463...|       7.0|
|            7.0|(30,[1,9,27,29],[...|[-3.3270713716617...|[2.49628391602761...|      12.0|
|            7.0|(30,[6,12,27,29],...|[-2.4814745490735...|[0.00121393536252...|       7.0|
|            7.0|(30,[7,14,27,29],...|[-2.7380997418824...|[3.83748937694834...|       7.0|
|            7.0|(30,[6,12,25,29],...|[-1.9060845952212...|[0.00195370490040...|

### Evaluation: Full Downsampled Training Set

In [37]:
accuracy = evaluator.evaluate(prediction_adj)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 20.062633262260128%
Test Error: 79.93736673773988%


Downsampling reduced accuracy to 20.1%, but this is due to the model not resorting mainly to the naive conclusion of the most frequent class.

In [38]:
prediction_adj.groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       8.0|  634|
|       0.0| 2483|
|       7.0| 1520|
|      18.0|  390|
|       1.0| 1447|
|       4.0|  392|
|      11.0|   96|
|      14.0| 1197|
|      19.0|  829|
|       3.0|  897|
|       2.0|  520|
|      17.0|  520|
|      10.0|  296|
|      13.0| 1065|
|       6.0|  246|
|      20.0|   12|
|      15.0|  679|
|       5.0|   42|
|       9.0|  412|
|      16.0|   50|
+----------+-----+
only showing top 20 rows



In [39]:
prediction_adj.count()

15008

In [40]:
2483 / 15008

0.16544509594882728

The predictions made for Circulatory Disease were reduced from 71.8% to 16.5% due to the downsampling.

## Validation Set

### Cleaning

In [41]:
val_set.show()

+----+---------+---+---+--------------+--------------------+---------+
| ucd|education|sex|age|marital_status|hispanic_race_recode|ucd_short|
+----+---------+---+---+--------------+--------------------+---------+
|A045|        3|  F| 87|             M|                   6|        A|
|A047|        1|  F| 74|             S|                   1|        A|
|A047|        1|  F| 83|             M|                   1|        A|
|A047|        1|  F| 96|             W|                   8|        A|
|A047|        1|  M| 74|             M|                   1|        A|
|A047|        1|  M| 80|             M|                   1|        A|
|A047|        1|  M| 81|             M|                   6|        A|
|A047|        1|  M| 90|             D|                   6|        A|
|A047|        2|  F| 74|             W|                   6|        A|
|A047|        2|  F| 80|             W|                   6|        A|
|A047|        2|  F| 82|             W|                   6|        A|
|A047|

In [42]:
val_set.groupby("ucd_short").count().sort('count').show()

+---------+-----+
|ucd_short|count|
+---------+-----+
|        H|   16|
|        O|  155|
|        L|  730|
|        Q| 1485|
|        P| 1609|
|        M| 2082|
|        Y| 2193|
|        B| 2506|
|        D| 4091|
|        R| 5019|
|        V| 6504|
|        W| 7555|
|        A| 8413|
|        N|10823|
|        K|16546|
|        E|19260|
|        X|19691|
|        F|21705|
|        G|31520|
|        J|41936|
+---------+-----+
only showing top 20 rows



In [43]:
# there are only 16 observations for group 'H' - filter out

val_set = val_set.filter(val_set.ucd_short != "H")

In [44]:
val_set.groupby("ucd_short").count().sort('count').show()

+---------+-----+
|ucd_short|count|
+---------+-----+
|        O|  155|
|        L|  730|
|        Q| 1485|
|        P| 1609|
|        M| 2082|
|        Y| 2193|
|        B| 2506|
|        D| 4091|
|        R| 5019|
|        V| 6504|
|        W| 7555|
|        A| 8413|
|        N|10823|
|        K|16546|
|        E|19260|
|        X|19691|
|        F|21705|
|        G|31520|
|        J|41936|
|        C|93591|
+---------+-----+
only showing top 20 rows



In [45]:
val_set.select('ucd_short').distinct().count()

21

### Downsampling

In [46]:
adj_val = downSample(df = val_set, target = 'ucd_short', seed = 212)

In [47]:
adj_val.groupby("ucd_short").count().sort('count').show()

+---------+-----+
|ucd_short|count|
+---------+-----+
|        Y|  136|
|        M|  141|
|        Q|  141|
|        L|  142|
|        G|  145|
|        B|  146|
|        P|  146|
|        O|  155|
|        J|  155|
|        I|  158|
|        X|  159|
|        E|  161|
|        V|  161|
|        K|  162|
|        F|  162|
|        D|  163|
|        R|  164|
|        N|  165|
|        A|  166|
|        W|  166|
+---------+-----+
only showing top 20 rows



### Modeling

In [48]:
# Full Model, Validation set, No downsampling

# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexed1_v = indexer1.fit(val_set).transform(val_set)
indexed2_v = indexer2.fit(indexed1_v).transform(indexed1_v)
indexed_v = indexer3.fit(indexed2_v).transform(indexed2_v)

# One-Hot Encoder
model_v = encoder.fit(indexed_v)
encoded_v = model_v.transform(indexed_v)

# VectorAssembler to package the features for model
output_v = assembler_full.transform(encoded_v)

# Cleanup
cleanup_v = output_v.select("ucd_short_index", "features")
cleanup_v.printSchema()

# Model
prediction_v = mlrModel_full.transform(cleanup_v)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [49]:
# Evaluation
accuracy = evaluator.evaluate(prediction_v)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 34.425038430449064%
Test Error: 65.57496156955094%


Accuracy: 34.4%

In [50]:
prediction_v.groupby("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|306690|
|      18.0|   165|
|       1.0| 96619|
|       4.0|   168|
|      11.0|     1|
|       3.0|    29|
|       2.0|     8|
|      17.0|  4416|
|       5.0| 19940|
|      12.0|    10|
+----------+------+



In [51]:
prediction_v.count()

428046

In [52]:
306690/428046

0.7164884147965406

71.65% of predictions were made for Circulatory Disease.  Will attempt downsampling and rerun the model to see if predictions are more evenly distributed.

In [53]:
# Full Downsampled Model, Validation set

# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexed1_adj_v = indexer1.fit(adj_val).transform(adj_val)
indexed2_adj_v = indexer2.fit(indexed1_adj_v).transform(indexed1_adj_v)
indexed_adj_v = indexer3.fit(indexed2_adj_v).transform(indexed2_adj_v)

# One-Hot Encoder
model_adj_v = encoder.fit(indexed_adj_v)
encoded_adj_v = model_adj_v.transform(indexed_adj_v)

# VectorAssembler to package the features for model
output_adj_v = assembler_full.transform(encoded_adj_v)

# Cleanup
cleanup_adj_v = output_adj_v.select("ucd_short_index", "features")
cleanup_adj_v.printSchema()

# Model
prediction_adj_v = mlrModel_adj.transform(cleanup_adj_v)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [54]:
prediction_adj_v.show()

+---------------+--------------------+--------------------+--------------------+----------+
|ucd_short_index|            features|       rawPrediction|         probability|prediction|
+---------------+--------------------+--------------------+--------------------+----------+
|           13.0|(30,[7,11,25,28,2...|[-1.4390660178380...|[0.00426361727316...|       1.0|
|           13.0|(30,[6,11,26,28,2...|[-1.3930541506783...|[0.00368007328808...|       1.0|
|           13.0|(30,[6,11,27,28,2...|[-2.8289250122771...|[5.84619912305389...|       1.0|
|           13.0|(30,[11,25,28,29]...|[-2.5139057527382...|[9.89735733437085...|      18.0|
|           13.0|(30,[6,13,25,28,2...|[-2.4774649872706...|[0.00108292326352...|      18.0|
|           13.0|(30,[13,25,28,29]...|[-2.3213748294455...|[0.00151655376328...|      18.0|
|           13.0|(30,[6,12,25,28,2...|[-2.3539775089541...|[9.94368220854813...|       1.0|
|           13.0|(30,[6,12,25,28,2...|[-2.4233876509772...|[8.92645856956415...|

In [55]:
# Evaluation
accuracy = evaluator.evaluate(prediction_adj_v)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 3.6491873658386997%
Test Error: 96.3508126341613%


Accuracy: 3.65%

Validation set accuracy, using downsampling, is 3.65% (much lower than the training set.  Overtrained.)

In [56]:
prediction_adj_v.groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       8.0|  220|
|       0.0|  460|
|       7.0|  366|
|      18.0|  102|
|       1.0|  256|
|       4.0|   57|
|      11.0|   49|
|      14.0|  223|
|      19.0|  173|
|       3.0|  215|
|       2.0|  123|
|      17.0|  206|
|      10.0|   68|
|      13.0|  178|
|       6.0|   59|
|      20.0|    4|
|      15.0|  103|
|       5.0|    7|
|       9.0|  122|
|      16.0|    3|
+----------+-----+
only showing top 20 rows



In [57]:
prediction_adj_v.count()

3261

In [58]:
460/3261

0.14106102422569763

Circulatory disease predictions: 14.11%

## Hyperparameter Tuning (On Downsampled Data)

### No Intercept model, training set

In [59]:
# Model
mlr_adj_noint = LogisticRegression(featuresCol = "features", labelCol = "ucd_short_index", family = "multinomial", fitIntercept = False)
mlrModel_adj_noint = mlr_adj_noint.fit(cleanup_adj)
prediction_adj_noint = mlrModel_adj_noint.transform(cleanup_adj)

In [60]:
accuracy = evaluator.evaluate(prediction_adj_noint)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 19.962686567164177%
Test Error: 80.03731343283582%


Accuracy: 20.0%

### No Intercept model, Validation set

In [61]:
# Model
prediction_adj_v_noint = mlrModel_adj_noint.transform(cleanup_adj_v)

In [62]:
# Evaluation
accuracy = evaluator.evaluate(prediction_adj_v_noint)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 3.833180006133088%
Test Error: 96.16681999386691%


Accuracy: 3.84%

### Including Elastic Net parameter, training set

In [63]:
# Model
mlr_adj_elast = LogisticRegression(featuresCol = "features", labelCol = "ucd_short_index", family = "multinomial", elasticNetParam = 0.5)
mlrModel_adj_elast = mlr_adj_elast.fit(cleanup_adj)
prediction_adj_elast = mlrModel_adj_elast.transform(cleanup_adj)

In [64]:
accuracy = evaluator.evaluate(prediction_adj_elast)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 20.062633262260128%
Test Error: 79.93736673773988%


Accuracy: 20.1%

### Including Elastic Net parameter, Validation set

In [65]:
# Model
prediction_adj_v_elast = mlrModel_adj_elast.transform(cleanup_adj_v)

In [66]:
accuracy = evaluator.evaluate(prediction_adj_v_elast)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 3.6491873658386997%
Test Error: 96.3508126341613%


Accuracy: 3.65%

### Variable Reduction to Age & Sex, training set

In [67]:
# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexer1 = StringIndexer(inputCol="sex", outputCol="sex_index")
indexer2 = StringIndexer(inputCol="ucd_short", outputCol="ucd_short_index")

indexed1_adj_r = indexer1.fit(train_set).transform(train_set)
indexed_adj_r = indexer2.fit(indexed1_adj_r).transform(indexed1_adj_r)

# VectorAssembler to package the features for model
assembler_adj_r = VectorAssembler(inputCols=["sex_index", "age"],
                            outputCol="features")
output_adj_r = assembler_adj_r.transform(indexed_adj_r)

# Cleanup
cleanup_adj_r = output_adj_r.select("ucd_short_index", "features")
cleanup_adj_r.printSchema()

# Model
mlr_adj_r = LogisticRegression(featuresCol = "features", labelCol = "ucd_short_index", family = "multinomial")
mlrModel_adj_r = mlr_adj_r.fit(cleanup_adj_r)
prediction_adj_r = mlrModel_adj_r.transform(cleanup_adj_r)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [68]:
accuracy = evaluator.evaluate(prediction_adj_r)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 33.04167142234202%
Test Error: 66.95832857765798%


Accuracy: 33.0%

### Variable Reduction to Age & Sex, Validation set

In [69]:
# Model
prediction_adj_v_r = mlrModel_adj_r.transform(cleanup_adj_r)

In [70]:
accuracy = evaluator.evaluate(prediction_adj_v_r)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 33.04167142234202%
Test Error: 66.95832857765798%


Accuracy: 33.04%

## Test Set

### Import and Cleaning

In [71]:
# Read in test data
test_data = spark.read.option("header",True).option("inferSchema",True).csv("test_data")

In [72]:
# Subset with potential features
testing = test_data.select("ucd", "education", "sex", "age", "marital_status", "hispanic_race_recode")

In [73]:
testing.printSchema()

root
 |-- ucd: string (nullable = true)
 |-- education: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- marital_status: string (nullable = true)
 |-- hispanic_race_recode: integer (nullable = true)



In [74]:
# strip UCD codes to include first letter only, for response level reduction
testing = testing.withColumn("ucd_short", regexp_replace('ucd', '\\d+', ''))

In [75]:
# replace age coding values
testing = testing.withColumn("age", replace(col("age"), 999)) # change age value coded as unknown (999) to null

In [76]:
testing.filter(testing.age == 999).count() # yes - 999 codes have been replaced 

0

In [77]:
testing.groupby("ucd_short").count().orderBy('count', ascending=False).show(5) # top 5 causes of death with counts

+---------+------+
|ucd_short| count|
+---------+------+
|        I|168944|
|        C|119737|
|        J| 54616|
|        G| 39785|
|        F| 27609|
+---------+------+
only showing top 5 rows



In [78]:
testing.count() # rows in training set

550650

In [79]:
168944/550650

0.3068083174430219

30.7% of the deaths in the test set are from circulatory disease.

In [80]:
# drop rows with null or nans
testing = testing.na.drop()

In [81]:
testing.count()

534741

In [82]:
(1 - (534741/550650))*100 # dropped 2.9% of dataset because of null values

2.889131026968128

In [83]:
testing.groupby("ucd_short").count().sort('count').show()

+---------+-----+
|ucd_short|count|
+---------+-----+
|        H|   21|
|        O|  209|
|        L|  951|
|        Q| 1745|
|        P| 1990|
|        Y| 2649|
|        M| 2729|
|        B| 3139|
|        D| 5207|
|        R| 6281|
|        V| 8049|
|        W| 9225|
|        A|10509|
|        N|13731|
|        K|20263|
|        E|23831|
|        X|24710|
|        F|26735|
|        G|39016|
|        J|53003|
+---------+-----+
only showing top 20 rows



In [84]:
# there are only 21 observations for group 'H' - filter out
testing = testing.filter(testing.ucd_short != "H")

In [85]:
testing.select('ucd_short').distinct().count()

21

In [90]:
testing.show(5)

+----+---------+---+---+--------------+--------------------+---------+
| ucd|education|sex|age|marital_status|hispanic_race_recode|ucd_short|
+----+---------+---+---+--------------+--------------------+---------+
| C61|        1|  M| 82|             M|                   7|        C|
|E142|        1|  F| 74|             M|                   1|        E|
|I429|        2|  M| 70|             S|                   8|        I|
|C349|        3|  M| 89|             W|                   6|        C|
|I119|        3|  M| 67|             S|                   7|        I|
+----+---------+---+---+--------------+--------------------+---------+
only showing top 5 rows



### Downsampling

In [86]:
adj_test = downSample(df = testing, target = 'ucd_short', seed = 212)

In [87]:
adj_test.groupby("ucd_short").count().sort('count').show()

+---------+-----+
|ucd_short|count|
+---------+-----+
|        L|  180|
|        P|  187|
|        B|  189|
|        Q|  190|
|        M|  190|
|        D|  191|
|        N|  194|
|        Y|  195|
|        C|  196|
|        X|  199|
|        V|  199|
|        K|  203|
|        F|  204|
|        E|  207|
|        I|  208|
|        O|  209|
|        J|  209|
|        A|  210|
|        W|  210|
|        R|  212|
+---------+-----+
only showing top 20 rows



### Modeling

In [94]:
# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexer1 = StringIndexer(inputCol="marital_status", outputCol="marital_status_index")
indexer2 = StringIndexer(inputCol="sex", outputCol="sex_index")
indexer3 = StringIndexer(inputCol="ucd_short", outputCol="ucd_short_index")

In [95]:
# Full Model, Test set, No downsampling

# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexed1_t = indexer1.fit(testing).transform(testing)
indexed2_t = indexer2.fit(indexed1_t).transform(indexed1_t)
indexed_t = indexer3.fit(indexed2_t).transform(indexed2_t)

# One-Hot Encoder
model_t = encoder.fit(indexed_t)
encoded_t = model_t.transform(indexed_t)

# VectorAssembler to package the features for model
output_t = assembler_full.transform(encoded_t)

# Cleanup
cleanup_t = output_t.select("ucd_short_index", "features")
cleanup_t.printSchema()

# Model
prediction_t = mlrModel_full.transform(cleanup_t)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [96]:
# Evaluation
accuracy = evaluator.evaluate(prediction_t)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 34.53134350688211%
Test Error: 65.46865649311789%


Accuracy: 34.53%

In [97]:
prediction_t.groupby("prediction").count().show()

+----------+------+
|prediction| count|
+----------+------+
|       0.0|383491|
|      18.0|   175|
|       1.0|120408|
|       4.0|   190|
|      11.0|     1|
|       3.0|    30|
|       2.0|     9|
|      17.0|  5442|
|       5.0| 24960|
|      12.0|    14|
+----------+------+



In [98]:
prediction_t.count()

534720

In [99]:
383491/534720

0.7171809545182526

71.72% of predictions were made for Circulatory Disease.  This is because the dataset is unbalanced.  Will attempt downsampling and rerun the model to see if predictions are more evenly distributed.

In [100]:
# Full Downsampled Model, Test set

# StringIndexer transformers to encode categorical features as numerical for vector assembly
indexed1_adj_t = indexer1.fit(adj_test).transform(adj_test)
indexed2_adj_t = indexer2.fit(indexed1_adj_t).transform(indexed1_adj_t)
indexed_adj_t = indexer3.fit(indexed2_adj_t).transform(indexed2_adj_t)

# One-Hot Encoder
model_adj_t = encoder.fit(indexed_adj_t)
encoded_adj_t = model_adj_t.transform(indexed_adj_t)

# VectorAssembler to package the features for model
output_adj_t = assembler_full.transform(encoded_adj_t)

# Cleanup
cleanup_adj_t = output_adj_t.select("ucd_short_index", "features")
cleanup_adj_t.printSchema()

# Model
prediction_adj_t = mlrModel_adj.transform(cleanup_adj_t)

root
 |-- ucd_short_index: double (nullable = false)
 |-- features: vector (nullable = true)



In [101]:
# Evaluation
accuracy = evaluator.evaluate(prediction_adj_t)

print('Accuracy: ' + str(accuracy*100) + '%')
print('Test Error: ' + str((1.0 - accuracy)*100) + '%')

Accuracy: 5.552907530981887%
Test Error: 94.44709246901812%


Accuracy: 5.55%

In [102]:
prediction_adj_t.groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|       8.0|  183|
|       0.0|  737|
|       7.0|  429|
|      18.0|  115|
|       1.0|  369|
|       4.0|   86|
|      11.0|   32|
|      14.0|  353|
|      19.0|  246|
|       3.0|  225|
|       2.0|  120|
|      17.0|  182|
|      10.0|   78|
|      13.0|  320|
|       6.0|   57|
|      20.0|    4|
|      15.0|  178|
|       5.0|    8|
|       9.0|  127|
|      16.0|    4|
+----------+-----+
only showing top 20 rows



In [103]:
prediction_adj_t.count()

4196

In [104]:
737/4196

0.17564346997140134

Circulatory disease predictions: 17.56%

## Pipelining

In [ ]:
# Pipelining
# from pyspark.ml.linalg import Vectors
# from pyspark.ml.feature import VectorAssembler
# from pyspark.ml.feature import OneHotEncoder, StringIndexer
# from pyspark.ml.feature import OneHotEncoderEstimator
# from pyspark.ml.classification import LogisticRegression
# from pyspark.ml import Pipeline

# # Configure an ML pipeline, which consists of 5 stages: StringIndexers(1-3), OneHotEncoder, VectorAssembler.  Logistic Regression object kept out of Pipeline for now

# indexer1 = StringIndexer(inputCol="marital_status", outputCol="marital_status_index")
# indexer2 = StringIndexer(inputCol="sex", outputCol="sex_index")
# indexer3 = StringIndexer(inputCol="ucd_short", outputCol="ucd_short_index")
# encoder = OneHotEncoderEstimator(inputCols =["hispanic_race_recode", "education", "marital_status_index"],
#                                  outputCols = ["cat_hispanic_race_recode", "cat_education", "cat_marital_status"])
# assembler = VectorAssembler(inputCols=["cat_hispanic_race_recode", "cat_education", "cat_marital_status", "sex_index", "age"],
#                             outputCol="features")
# multinomialRegression = LogisticRegression(featuresCol = "features", labelCol = "ucd_short_index", family = "multinomial")

# pipeline = Pipeline(stages=[indexer1, indexer2, indexer3, encoder, assembler, multinomialRegression])

In [ ]:
# multinomialModel = pipeline.fit(train_set)

In [ ]:
# transformed_train = multinomialModel.transform(train_set)